In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from gensim import corpora, models, similarities
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
conn = MongoClient()
db = conn.project

In [3]:
db.recipes.count()

86

### Vectorize Recipes and Prepare for Similarity Queries

In [5]:
cursor = db.recipes.find({}, {'title': 1, 'ingredients': 1, '_id' : 0})
for doc in cursor:
    print doc
#     print doc['title']
#     print "; ".join(doc['ingredients'])

{u'ingredients': [u'1 small shallot, chopped', u'2 tablespoons chopped flat-leaf parsley', u'2 tablespoons red wine vinegar', u'2 teaspoons fresh thyme leaves', u'Kosher salt, freshly ground pepper', u'1/3 cup extra-virgin olive oil', u'2 cups cooked basmati rice, cooled', u'2 cups bite-size pieces assorted vegetables (such as radishes, tomatoes, peas, summer squash) or carrot ribbons', u'3/4 cup torn mixed leafy greens, sprouts, and herbs', u'1/3 cup chopped red, yellow, or white onion or scallions', u'2 tablespoons toasted pine nuts (optional)'], u'title': u'Basmati Rice with Summer Vegetable Salad'}
{u'ingredients': [u'2 pounds new potatoes, halved', u'1 lemon, thinly sliced', u'2 tablespoons olive oil', u'2 tablespoons kosher salt and black pepper\n'], u'title': u'Roasted Potatoes and Lemon With Dill'}
{u'ingredients': [u'1/2 red onion, thinly sliced', u'About 1 1/2 cups red wine vinegar', u'1/4 cup olive oil', u'1 1/2 teaspoons ancho chile powder', u'1 1/2 teaspoons dried oregano'

In [4]:
cursor = db.recipes.find({}, {'title': 1, 'ingredients': 1, '_id' : 0})
data = pd.DataFrame(list(cursor))

In [7]:
data.head()

,ingredients,title
0,"[1 small shallot, chopped, 2 tablespoons chopp...",Basmati Rice with Summer Vegetable Salad
1,"[2 pounds new potatoes, halved, 1 lemon, thinl...",Roasted Potatoes and Lemon With Dill
2,"[1/2 red onion, thinly sliced, About 1 1/2 cup...",Fish Tacos
3,"[6 tablespoons Asian fish sauce, 6 tablespoons...",cold rice noodles with peanut-lime chicken
4,[1 recipe pastry for a 9 inch double crust pie...,Apple Pie by Grandma Ople


In [5]:
data['ingredients'] = data['ingredients'].apply(lambda x: " ".join(x))

In [6]:
data['ingredients']

0     1 small shallot, chopped 2 tablespoons chopped...
1     2 pounds new potatoes, halved 1 lemon, thinly ...
2     1/2 red onion, thinly sliced About 1 1/2 cups ...
3     6 tablespoons Asian fish sauce 6 tablespoons b...
4     1 recipe pastry for a 9 inch double crust pie ...
5     1 package 1 package 1 cup 1 cup 1 whole 1 whol...
6     1 1/4 cups all-purpose flour 1/4 teaspoon salt...
7     2 tablespoons olive oil 1 large yellow onion, ...
8     1 15- to 15 1/2-ounce can chickpeas (garbanzo ...
9     1 cup Almond Breeze Unsweetened Almondmilk Coc...
10    100g pancetta 50g pecorino cheese 50g Parmesan...
11     cups  cups  cups  cups  teaspoons  teaspoons ...
12    FOR THE CAKE: 1 cup Prunes 1 cup Sugar 3 whole...
13    1 1/2 pounds cherry tomatoes or other small to...
14    1 whole 1 whole 2 cloves 2 cloves 2 cans 2 can...
15    2 cups cabbage, finely shredded 1 cup leeks, w...
16    1 pound fresh asparagus spears, trimmed 1 tabl...
17    2 tsp olive oil 1 onion , finely chopped 1

In [7]:
documents = data['ingredients'].values

In [11]:
documents[:2]

array([ u'1 small shallot, chopped 2 tablespoons chopped flat-leaf parsley 2 tablespoons red wine vinegar 2 teaspoons fresh thyme leaves Kosher salt, freshly ground pepper 1/3 cup extra-virgin olive oil 2 cups cooked basmati rice, cooled 2 cups bite-size pieces assorted vegetables (such as radishes, tomatoes, peas, summer squash) or carrot ribbons 3/4 cup torn mixed leafy greens, sprouts, and herbs 1/3 cup chopped red, yellow, or white onion or scallions 2 tablespoons toasted pine nuts (optional)',
       u'2 pounds new potatoes, halved 1 lemon, thinly sliced 2 tablespoons olive oil 2 tablespoons kosher salt and black pepper\n'], dtype=object)

In [15]:
documents[0]

u'1 small shallot, chopped 2 tablespoons chopped flat-leaf parsley 2 tablespoons red wine vinegar 2 teaspoons fresh thyme leaves Kosher salt, freshly ground pepper 1/3 cup extra-virgin olive oil 2 cups cooked basmati rice, cooled 2 cups bite-size pieces assorted vegetables (such as radishes, tomatoes, peas, summer squash) or carrot ribbons 3/4 cup torn mixed leafy greens, sprouts, and herbs 1/3 cup chopped red, yellow, or white onion or scallions 2 tablespoons toasted pine nuts (optional)'

In [17]:
list_ = [[word for word in document.lower().split() if not any(c.isdigit() for c in word)] for document in documents]
print list_[0]

[u'small', u'shallot,', u'chopped', u'tablespoons', u'chopped', u'flat-leaf', u'parsley', u'tablespoons', u'red', u'wine', u'vinegar', u'teaspoons', u'fresh', u'thyme', u'leaves', u'kosher', u'salt,', u'freshly', u'ground', u'pepper', u'cup', u'extra-virgin', u'olive', u'oil', u'cups', u'cooked', u'basmati', u'rice,', u'cooled', u'cups', u'bite-size', u'pieces', u'assorted', u'vegetables', u'(such', u'as', u'radishes,', u'tomatoes,', u'peas,', u'summer', u'squash)', u'or', u'carrot', u'ribbons', u'cup', u'torn', u'mixed', u'leafy', u'greens,', u'sprouts,', u'and', u'herbs', u'cup', u'chopped', u'red,', u'yellow,', u'or', u'white', u'onion', u'or', u'scallions', u'tablespoons', u'toasted', u'pine', u'nuts', u'(optional)']


In [18]:
any(c.isdigit() for c in '30ml50ml')

True

In [28]:
import string
string.punctuation


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [30]:
wnl = WordNetLemmatizer()
stopset = set(stopwords.words('english'))

list_lem = [[wnl.lemmatize(word.strip(string.punctuation)) for word in document.lower().replace("/", "").split()\
             if word not in stopset if not word.isdigit()] for document in documents]
print list_lem[0]

[u'small', u'shallot', u'chopped', u'tablespoon', u'chopped', u'flat-leaf', u'parsley', u'tablespoon', u'red', u'wine', u'vinegar', u'teaspoon', u'fresh', u'thyme', u'leaf', u'kosher', u'salt', u'freshly', u'ground', u'pepper', u'cup', u'extra-virgin', u'olive', u'oil', u'cup', u'cooked', u'basmati', u'rice', u'cooled', u'cup', u'bite-size', u'piece', u'assorted', u'vegetable', u'such', u'radish', u'tomato', u'pea', u'summer', u'squash', u'carrot', u'ribbon', u'cup', u'torn', u'mixed', u'leafy', u'green', u'sprout', u'herb', u'cup', u'chopped', u'red', u'yellow', u'white', u'onion', u'scallion', u'tablespoon', u'toasted', u'pine', u'nut', u'optional']


In [31]:
def clean_text(documents):
    '''
    INPUT: array of strings
    OUTPUT: array of lists (ok?)
    '''
    stopset = set(stopwords.words('english'))
    stopset.update(['description', 'available']) ## add some words that appear a lot in menu data
    wnl = WordNetLemmatizer()
    texts = [[wnl.lemmatize(word.strip(string.punctuation)) for word in document.lower().replace("/", "").split()\
             if word not in stopset if not word.isdigit()] for document in documents]
    text_array = np.array(texts)
    return text_array

In [32]:
text_array = clean_text(documents)

In [33]:
text_array[0]

[u'small',
 u'shallot',
 u'chopped',
 u'tablespoon',
 u'chopped',
 u'flat-leaf',
 u'parsley',
 u'tablespoon',
 u'red',
 u'wine',
 u'vinegar',
 u'teaspoon',
 u'fresh',
 u'thyme',
 u'leaf',
 u'kosher',
 u'salt',
 u'freshly',
 u'ground',
 u'pepper',
 u'cup',
 u'extra-virgin',
 u'olive',
 u'oil',
 u'cup',
 u'cooked',
 u'basmati',
 u'rice',
 u'cooled',
 u'cup',
 u'bite-size',
 u'piece',
 u'assorted',
 u'vegetable',
 u'such',
 u'radish',
 u'tomato',
 u'pea',
 u'summer',
 u'squash',
 u'carrot',
 u'ribbon',
 u'cup',
 u'torn',
 u'mixed',
 u'leafy',
 u'green',
 u'sprout',
 u'herb',
 u'cup',
 u'chopped',
 u'red',
 u'yellow',
 u'white',
 u'onion',
 u'scallion',
 u'tablespoon',
 u'toasted',
 u'pine',
 u'nut',
 u'optional']

In [9]:
dictionary = corpora.Dictionary(texts)

In [11]:
count = 0
for i in dictionary.iteritems():
    count +=1

In [12]:
print count

1018


In [16]:
 print(dictionary.token2id)

{u'squid,': 970, u'monterey': 487, u'serranos,': 196, u'chile': 93, u'jonagold': 412, u'(1/4': 548, u'ale': 468, u'hot': 784, u'yellow': 199, u'baby': 948, u'freeze-dried': 401, u'raspberries,': 471, u'asian': 117, u'(if': 658, u'parmesan': 255, u'inch)': 605, u'barbecue': 932, u'mascarpone': 434, u'paprika,': 712, u'milk': 407, u'quartered': 730, u'blueberries': 638, u'/': 507, u'peanut': 127, u'microwave': 907, u'sparkling': 474, u'unflavored': 467, u'pancetta': 272, u'oz': 617, u'dairy': 945, u'teaspoon': 87, u'dressing': 904, u'whisky': 394, u'brown': 145, u'quinoa': 819, u'doz': 850, u'yeast': 923, u'sage': 837, u'thawed': 470, u'fat': 578, u'tacos': 860, u'powder,': 428, u'farro': 225, u'butterolive': 616, u'jack': 604, u'softened': 533, u'ladyfingers': 433, u'ounces,': 286, u'(mine': 924, u'ounces)': 636, u'room': 846, u'cooks': 961, u'tortillas': 97, u'baking': 290, u'nectar': 535, u'asparagus': 335, u'pinch': 165, u'sticks)': 529, u'pecorino': 278, u'large': 238, u'slices,': 9

In [27]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [18]:
corpus[:2]

[[(0, 3),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 2),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 3),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 6),
  (25, 1),
  (26, 3),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 2),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1)],
 [(14, 1),
  (24, 3),
  (26, 2),
  (33, 1),
  (34, 1),
  (43, 1),
  (44, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1)]]

In [19]:
tfidf = models.TfidfModel(corpus)
index = similarities.Similarity(tfidf[corpus]) ## prepare for similarity queries

### Get a Restaurant to Search 

In [15]:
cursor_test = db.restaurants.find({}, {'name':1, '_id' :0}).limit(10)

In [16]:
for i in cursor_test:
    print i

{u'name': u'Hogwash'}
{u'name': u'Fog Harbor Fish House'}
{u'name': u'Marlowe'}
{u'name': u'Gary Danko'}
{u'name': u'SO'}
{u'name': u'Chez Maman'}
{u'name': u'HRD'}
{u'name': u'The Italian Homemade Company'}
{u'name': u'Lord George'}
{u'name': u'Causwells'}


In [20]:
r_cursor = db.restaurants.find_one({'name' : 'Nopa'})

In [21]:
menu = r_cursor['menu']

In [22]:
menu_list = []
for i in zip(menu['items'], menu['descriptions']):
    menu_list.append(': '.join(i))

In [23]:
menu_string = ' '.join(menu_list)

In [24]:
menu_string

u'Warm Marinated Olives: no description available Chicken Soup: Farro, Spinach and Sage Grilled Calamari: Cauliflower, Olives, Fris\xe9e and A\xcfoli Warm Goat Cheese: Grapes, Persimmon and Crostini Manila Clams: Toasted Fregola, Peppers, Lemon and Parsley Olive Oil Poached Albacore: Marinated Cucumbers, Romesco and Beets Flatbread of House smoked Bacon: Potatoes, Cheddar and Horseradish Little Gem Lettuces: Warren Pear, Blue Cheese, Hazelnuts and Medjool Dates Tasso Spiced Ham: Butternut Squash, Sherry Onions, Almonds, Balsamic and Poached Egg Avocado Toast: Capers, Pickled Jalape\xf1o, Arugula and Shaved Pantaleo Wood Baked Butter Beans: Feta, Oregano Pesto and Breadcrumbs Duck Liver Mousse: Confit Gizzard, Pluots and Grilled Bread Crispy Mushrooms and Jimmy Nardello Pepper Sauce: no description available Moroccan Vegetable Tagine: Toasted Almonds and Lemon Yogurt Country Pork Chop: Arrowhead Cabbage, Potatoes, Apple and Mustard Grilled Leg of Lamb: Toasted Quinoa, Delicata Squash, F

In [25]:
menu_tokens = [word for word in menu_string.lower().split() if word not in stoplist]  #add description,  available to stopwords?
menu_vector = dictionary.doc2bow(menu_tokens)

In [ ]:
sims = index[tfidf[menu_vector]]  ## kills kernel :(  Try running from terminal?
## will argmax to get position and refer back to recipes df to get title.

In [19]:
type(stoplist)

set

In [20]:
stoplist.add('description')

In [24]:
stoplist.update(['description', 'available'])

In [25]:
stoplist

{u'a',
 u'about',
 u'above',
 u'after',
 u'again',
 u'against',
 u'ain',
 u'all',
 u'am',
 u'an',
 u'and',
 u'any',
 u'are',
 u'aren',
 u'as',
 u'at',
 'available',
 u'be',
 u'because',
 u'been',
 u'before',
 u'being',
 u'below',
 u'between',
 u'both',
 u'but',
 u'by',
 u'can',
 u'couldn',
 u'd',
 'description',
 u'did',
 u'didn',
 u'do',
 u'does',
 u'doesn',
 u'doing',
 u'don',
 u'down',
 u'during',
 u'each',
 u'few',
 u'for',
 u'from',
 u'further',
 u'had',
 u'hadn',
 u'has',
 u'hasn',
 u'have',
 u'haven',
 u'having',
 u'he',
 u'her',
 u'here',
 u'hers',
 u'herself',
 u'him',
 u'himself',
 u'his',
 u'how',
 u'i',
 u'if',
 u'in',
 u'into',
 u'is',
 u'isn',
 u'it',
 u'its',
 u'itself',
 u'just',
 u'll',
 u'm',
 u'ma',
 u'me',
 u'mightn',
 u'more',
 u'most',
 u'mustn',
 u'my',
 u'myself',
 u'needn',
 u'no',
 u'nor',
 u'not',
 u'now',
 u'o',
 u'of',
 u'off',
 u'on',
 u'once',
 u'only',
 u'or',
 u'other',
 u'our',
 u'ours',
 u'ourselves',
 u'out',
 u'over',
 u'own',
 u're',
 u's',
 u'same

## Using PreTrained Word2Vec

Pretrained Word2vec model from GloVe: Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation.
OR try GoogleNews- trained vectors from Google https://code.google.com/archive/p/word2vec/

In [ ]:
## Load the model
#model = models.Word2Vec.load_word2vec_format('models/glove.6B.50d.txt', binary=False)
# fn = "models/GoogleNews-vectors-negative300.bin.gz" # pretrained on Google News
# model = models.Word2Vec.load_word2vec_format(fn, binary= True) ## takes too long..

In [ ]:
## Convert words to vectors
## For restaurants texts
# menu_vector = model[word] # not by word but by doc?
# ## For recipe texts
# index = similarities.Similarity(model[corpus]) ## corpus or texts?


In [ ]:
## Find most similar 
# sims = index[model[menu_vector]] ## convert BOW to Tfidf
# rec_indices = np.argsort(sims)[:-num:-1] # gets top 10 ## do I want to report similarity score too?
# data.loc[rec_indices, 'title'], sims[rec_indices]

In [ ]:
# print model.most_similar('vacation')